<a href="https://colab.research.google.com/github/beercafeguy/PyMLLab/blob/master/daily_learnings/hyperparam_hyperopt_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Data

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle

In [4]:
import json
kaggle_api = {
    "username": "codepandablog",
    "key": "##########################"
}

with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_api, f)

# Change the permission of the file to secure it
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d adityakadiwal/water-potability

Dataset URL: https://www.kaggle.com/datasets/adityakadiwal/water-potability
License(s): CC0-1.0
  0% 0.00/251k [00:00<?, ?B/s]
100% 251k/251k [00:00<00:00, 77.1MB/s]


In [6]:
!unzip water-potability.zip

Archive:  water-potability.zip
  inflating: water_potability.csv    


# Imports

In [1]:
# read in packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load Data

In [7]:
data = pd.read_csv('water_potability.csv')
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [8]:
data = data.dropna()
data.info()
data

<class 'pandas.core.frame.DataFrame'>
Index: 2011 entries, 3 to 3271
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2011 non-null   float64
 1   Hardness         2011 non-null   float64
 2   Solids           2011 non-null   float64
 3   Chloramines      2011 non-null   float64
 4   Sulfate          2011 non-null   float64
 5   Conductivity     2011 non-null   float64
 6   Organic_carbon   2011 non-null   float64
 7   Trihalomethanes  2011 non-null   float64
 8   Turbidity        2011 non-null   float64
 9   Potability       2011 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 172.8 KB


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
...,...,...,...,...,...,...,...,...,...,...
3267,8.989900,215.047358,15921.412018,6.297312,312.931022,390.410231,9.899115,55.069304,4.613843,1
3268,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1
3269,11.491011,94.812545,37188.826022,9.263166,258.930600,439.893618,16.172755,41.558501,4.369264,1
3270,6.069616,186.659040,26138.780191,7.747547,345.700257,415.886955,12.067620,60.419921,3.669712,1


In [9]:
data.shape

(2011, 10)

In [10]:
data = data.dropna()
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
...,...,...,...,...,...,...,...,...,...,...
3267,8.989900,215.047358,15921.412018,6.297312,312.931022,390.410231,9.899115,55.069304,4.613843,1
3268,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1
3269,11.491011,94.812545,37188.826022,9.263166,258.930600,439.893618,16.172755,41.558501,4.369264,1
3270,6.069616,186.659040,26138.780191,7.747547,345.700257,415.886955,12.067620,60.419921,3.669712,1


# Split the data

In [18]:
X = data.drop(['Potability'], axis = 1)
y = data['Potability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=200)

# Build initial model using a random forest

In [19]:
%time
model = RandomForestClassifier(n_estimators=300, max_features='sqrt', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


# View the performance

In [20]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6724565756823822


In [16]:
!pip install hyperopt

# Read Hyperopt values

In [17]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

tpe is the Bayesian algorithm in hyperopt for optimization

# Objective Function

In [28]:
# prompt: Write a objective function for hyperopt tpe which is maximizing accuracy

# objective function : Here we are negating accuracy because we want to increase the accuracy
# and hyperopt tries to minimize the loss
def objective(n_estimators):
    model = RandomForestClassifier(n_estimators=n_estimators, max_features='sqrt', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK}


# Set the hyperparam tuning algorithm

In [23]:
algorithm=tpe.suggest

# Define the values to search over for n_estimators

In [24]:
search_space = hp.randint('n_estimators', 200, 1000)

# Run the tuning process

In [29]:
%time
best_params = fmin(fn=objective,
                   space=search_space,
                   algo=algorithm,
                   max_evals=10)


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs
Accuracy:
0.6774193548387096
Accuracy:
0.674937965260546
Accuracy:
0.6650124069478908
Accuracy:
0.6774193548387096
Accuracy:
0.6799007444168734
Accuracy:
0.6724565756823822
Accuracy:
0.6674937965260546
Accuracy:
0.6699751861042184
Accuracy:
0.6774193548387096
Accuracy:
0.674937965260546
100%|██████████| 10/10 [00:59<00:00,  5.98s/trial, best loss: -0.6799007444168734]


# Result

In [27]:
best_params

{'n_estimators': 339}

# Hyperopt using multiple params

## Objective Functions

In [34]:
# Redefine the function to include wider range of hyperparameters
def objective(search_space):
    model = RandomForestClassifier(**search_space, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print({'loss': -accuracy, 'status': STATUS_OK, 'search_space': search_space})
    return {'loss': -accuracy, 'status': STATUS_OK}


## Search Space

In [32]:
# Define the search space ove
search_space = {'n_estimators': hp.randint('n_estimators', 200, 1000),
                'max_depth': hp.randint('max_depth',10,200),
                'min_samples_split': hp.uniform('min_samples_split',0,1),
                'min_samples_leaf': hp.randint('min_samples_leaf',1,10),
                'criterion' : hp.choice('criterion',['gini', 'entropy']),
                'max_features' : hp.choice('max_features',['sqrt', 'log2'])}

## Evaluate

In [35]:
best_params = fmin(
  fn=objective,
  space=search_space,
  algo=algorithm,
  max_evals=10)

{'loss': -0.6104218362282878, 'status': 'ok', 'search_space': {'criterion': 'gini', 'max_depth': 64, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 0.5126889637236957, 'n_estimators': 947}}
{'loss': -0.6153846153846154, 'status': 'ok', 'search_space': {'criterion': 'entropy', 'max_depth': 70, 'max_features': 'log2', 'min_samples_leaf': 8, 'min_samples_split': 0.3886104132259319, 'n_estimators': 829}}
{'loss': -0.652605459057072, 'status': 'ok', 'search_space': {'criterion': 'gini', 'max_depth': 152, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 0.08737672055880852, 'n_estimators': 539}}
{'loss': -0.5880893300248139, 'status': 'ok', 'search_space': {'criterion': 'gini', 'max_depth': 139, 'max_features': 'sqrt', 'min_samples_leaf': 9, 'min_samples_split': 0.8307582070903592, 'n_estimators': 831}}
{'loss': -0.5880893300248139, 'status': 'ok', 'search_space': {'criterion': 'entropy', 'max_depth': 191, 'max_features': 'sqrt', 'min_samples_leaf': 1,

# Results

In [37]:
space_eval(search_space, best_params)

{'criterion': 'gini',
 'max_depth': 152,
 'max_features': 'log2',
 'min_samples_leaf': 5,
 'min_samples_split': 0.08737672055880852,
 'n_estimators': 539}